In [84]:
import utils
import pandas as pd

Load the data and the ideal models for modulation and tonicization

In [85]:
df, dfm, dft = utils.load_dataset('all_files')

Load the data from the `justkeydding` predictions

In [86]:
dfjk = utils.justkeydding_to_dfdictionary('run_justkeydding.tsv')

Decide the evaluation parameters

In [140]:
# Model to evaluate
model = dfjk
# Task to evaluate
task_dict = {
    'modulation': 'local_key_label', 
    'tonicization': 'tonicized_key_label'
}
task = task_dict['modulation']

Make a sanity check that verifies that the index of the predictions matches the index of the ground truth annotations

In [141]:
errors = 0
for f in list(df.keys()):
    if not df[f].index.equals(dfjk[f].index):
        print(f"Warning. Index of file {f} doesn't match the index of ground truth")
        print(df[f].index.difference(dfjk[f].index))
        errors += 1
if errors == 0:
    print('Nice! No errors in the indexes!')

Nice! No errors in the indexes!


If no errors, start the evaluation

In [142]:
def score_slice(evaldfrow):
    gt, pred = evaldfrow
    score = utils.score_key_prediction(gt, pred)
    return score

In [143]:
scoredict = {}
for f in list(df.keys()):
    # Create the evaluation dataframe
    dfeval = pd.DataFrame(index=df[f].index)
    dfeval['gt'] = df[f][task]
    dfeval['pred'] = model[f].local_key_label    
    dfeval['score'] = dfeval.apply(score_slice, axis=1, raw=True)
    score = dfeval['score'].sum() / dfeval.index.size
    scoredict[f] = score

In [144]:
scoredf = pd.DataFrame(scoredict.values())
print(scoredf.mean(), scoredf.std(), scoredf.min(), scoredf.max())

0    0.548062
dtype: float64 0    0.235816
dtype: float64 0    0.033333
dtype: float64 0    1.0
dtype: float64
